In [3]:
from operator import index
from turtle import clone
from selenium import webdriver # type: ignore
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pyperclip
import time
import pandas as pd
import os
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementClickInterceptedException, StaleElementReferenceException

# ✅ 브라우저 창 크기 설정
WINDOW_WIDTH = 1100
WINDOW_HEIGHT = 1000

# ✅ 브라우저 초기화 및 로그인
def setup_browser(position_x):
    driver = webdriver.Chrome()
    driver.set_window_size(WINDOW_WIDTH, WINDOW_HEIGHT)
    driver.set_window_position(position_x, 0)
    driver.get('https://console.thebackend.io/ko/login')
    time.sleep(0.2)
    return driver

# ✅ 로그인 함수
def login_to_console(driver):
    try:
        id_input = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "username")))
        pyperclip.copy("dksrufp0607@naver.com")
        id_input.send_keys(Keys.CONTROL, 'v')

        pw_input = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "password")))
        pyperclip.copy("ghkfkd159!")
        pw_input.send_keys(Keys.CONTROL, 'v')
        time.sleep(1)
        login_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button[type="submit"]')))
        login_button.click()
        print("✅ 로그인 완료!")
        time.sleep(2)
    except Exception as e:
        print(f"⚠️ 로그인 실패: {e}")

# ✅ 콘솔 페이지 이동
def navigate_to_page(driver):
    article_url = 'https://console.thebackend.io/ko/project/1ea3f14d34e89530ea88b3245bc82dc17d5f52ce1554049f19fce9219a847cfce18bb8891c9ffe90bc65e2b9a3b981853fc5513c1dd200afc9590ba6bfd5fced4230647d25328849e0917641/baseGameInfo/data'
    driver.get(article_url)
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
    print("✅ 페이지 로드 완료!")

# ✅ "확인" 버튼 클릭
def confirm_click(driver):
    print("🔍 '확인' 버튼 탐색 중...")
    try:
        confirm_button = WebDriverWait(driver, 3).until(
            EC.element_to_be_clickable((By.XPATH, '//button[contains(text(), "확인")]'))
        )
        confirm_button.click()
        print("✅ 확인 버튼 클릭 완료!")
    except Exception as e:
        print(f"⚠️ 확인 버튼을 찾거나 클릭하지 못했습니다: {e}")
        print("⏩ 그래도 계속 진행합니다.")

# ✅ CSV 데이터 로드 함수 (검색용)
def get_csv_value(row, col):
    global csv_path
    global name
    global stage
    try:
        df = pd.read_csv(csv_path)
        if row < df.shape[0] and col < df.shape[1]:
            print(f"🔹 {row}행 3열 값 (출력용): {df.iloc[row, 2]}")
            name = df.iloc[row, 2]
            stage = df.iloc[row, 3]
            return df.iloc[row, col]
        print("⚠️ CSV 파일의 크기를 초과하는 인덱스입니다.")
        return None
    except FileNotFoundError:
        print(f"⚠️ CSV 파일을 찾을 수 없습니다: {csv_path}")
        return None
    except Exception as e:
        print(f"⚠️ CSV 로드 중 오류 발생: {e}")
        return None

# ✅ 검색 실행 함수
def execute_tasks(driver, row, col, type):
    try:
        get_csv_value(row, col)
        global b2_value
        if not b2_value:
            print("⚠️ CSV에서 값을 가져오지 못했습니다.")
            return

        table_dropdown = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//div[contains(@class, "_manageInfo-data_1rntr_71")]')))
        table_dropdown.click()
        print("✅ 테이블 드롭다운 클릭 완료!")

        base_data_option = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, f'//div[@role="option"][span[text()="{type}"]]')))
        base_data_option.click()

        print("✅ BASE_DATA 선택 완료!")

        detail_search_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//p[contains(text(), "상세 검색")]')))
        time.sleep(3)
        try:
            detail_search_button.click()
        except ElementClickInterceptedException:
            driver.execute_script("arguments[0].click();", detail_search_button)
            print("⚠️ '상세 검색' 버튼 클릭 차단됨, JavaScript로 실행")
        print("✅ 상세 검색 클릭 완료!")

        # CSS 셀렉터로 드롭다운 요소를 대기하며 찾고 클릭
        dropdown = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, "#root > div.css-15qeme7 > div._context_1j77n_1.css-13sph2o > form > div._fixwidth_1rntr_1 > div.accordion.ui._accordion_1rntr_92 > div.content.active > div:nth-child(1) > div.ui.selection.dropdown"))
        )
        dropdown.click()
        print("✅ 대상 클릭 완료!")
        # <span class="text">유저 아이디</span> 요소를 대기하며 찾고 클릭
        target_span = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//span[@class="text" and text()="유저 아이디"]'))
        )
        target_span.click()

        search_input = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.NAME, "defaultSearchValue")))
        search_input.clear()
        search_input.send_keys(b2_value)
        print(f"✅ B2 값 입력 완료: {b2_value}")

        search_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//button[contains(@class, "ui medium positive button") and @type="submit"]')))
        search_button.click()
        print("✅ 검색 버튼 클릭 완료!")


        # 요소가 클릭 가능할 때까지 최대 10초 대기 후 클릭
        uuid_element = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//*[@id="gamer_id"]/p'))
        )
        uuid_element.click()

        print("✅ UUID 클릭 완료!")
    except Exception as e:
        print(f"⚠️ 검색 작업 중 오류 발생: {e}")

# ✅ 엑셀에서 데이터 로드 함수
def load_excel_data(filename):
    try:
        df = pd.read_excel(filename)
        data = df[["Column Name", "Data Type", "Value"]].values.tolist()
        print(f"✅ 엑셀 데이터 로드 완료! 경로: {os.path.abspath(filename)}")
        return data
    except FileNotFoundError:
        print(f"⚠️ 엑셀 파일을 찾을 수 없습니다: {filename}")
        return []
    except Exception as e:
        print(f"⚠️ 엑셀 파일 로드 중 오류 발생: {e}")
        return []

# ✅ JSON 데이터 입력을 위한 Ace Editor 값 설정 함수
def set_ace_editor_value(driver, ace_element, new_value):
    try:
        WebDriverWait(driver, 10).until(lambda d: d.execute_script("return typeof ace !== 'undefined' && ace.edit(arguments[0]) !== null", ace_element))
        driver.execute_script("arguments[0].scrollIntoView(true);", ace_element)
        driver.execute_script("""
            var editor = ace.edit(arguments[0]);
            editor.setValue(arguments[1], -1);
            editor.session.getUndoManager().reset();
        """, ace_element, str(new_value))
        updated_value = driver.execute_script("return ace.edit(arguments[0]).getValue();", ace_element).strip()
        print(f"✅ Ace Editor 값 설정 완료: {updated_value}")
    except Exception as e:
        print(f"⚠️ Ace Editor 값 설정 실패: {e}")
    


# ✅ 데이터 붙여넣기 함수 (컬럼명만 비교하고 값만 변경)
def paste_data(driver, data):
    try:
        # 수정 버튼 클릭 및 화면 축소
        edit_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//button[contains(@class, "ui mini primary button") and text()="수정"]')))
        edit_button.click()
        print("✅ 수정 버튼 클릭 완료!")
        time.sleep(1)
        driver.execute_script("document.body.style.zoom='25%'")
        print("✅ 웹 화면 25% 축소 완료!")
        time.sleep(2)

        # iframe 또는 기본 컨텍스트 확인
        driver.switch_to.default_content()
        context_found = False
        try:
            WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CLASS_NAME, "_columnBorder-edit-modal_1rntr_47")))
            print("🔍 기본 콘텐츠에서 요소 발견!")
            context_found = True
        except TimeoutException:
            iframes = driver.find_elements(By.TAG_NAME, "iframe")
            for i, frame in enumerate(iframes):
                driver.switch_to.frame(frame)
                try:
                    WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CLASS_NAME, "_columnBorder-edit-modal_1rntr_47")))
                    print(f"🔍 iframe {i}에서 요소 발견!")
                    context_found = True
                    break
                except TimeoutException:
                    driver.switch_to.default_content()
            if not context_found:
                print("⚠️ 모든 컨텍스트에서 요소를 찾지 못함")
                return

        # 한 번만 요소를 가져와서 매핑
        elements = WebDriverWait(driver, 15).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "_columnBorder-edit-modal_1rntr_47")))
        print(f"✅ 찾은 요소 개수: {len(elements)}")
        
        # 컬럼 이름과 요소를 딕셔너리로 매핑
        column_element_map = {}
        for idx, element in enumerate(elements):
            try:
                col_input = element.find_element(By.CSS_SELECTOR, 'input[type="text"][name$=".columnName"]')
                current_col_name = col_input.get_attribute("value")
                column_element_map[current_col_name] = element
                print(f"🔹 매핑: {current_col_name} -> 요소 {idx+1}")
            except NoSuchElementException:
                print(f"⚠️ 요소 {idx+1}에서 컬럼 이름 찾기 실패")
        
        # 처리된 컬럼 추적
        processed_columns = set()
        total_columns_to_process = len(data)
        
        # 엑셀 데이터 처리
        for col_name, _, new_value in data:
            if col_name in processed_columns:
                print(f"ℹ️ {col_name} 이미 처리됨, 건너뜀")
                continue

            print(f"🔍 처리 중: {col_name}")
            if col_name in column_element_map:
                element = column_element_map[col_name]
                try:
                    driver.execute_script("arguments[0].scrollIntoView(true);", element)
                    try:
                        ace_editor = element.find_element(By.CSS_SELECTOR, "div.ace_editor")
                        driver.execute_script("arguments[0].click();", ace_editor)
                        set_ace_editor_value(driver, ace_editor, new_value)
                    except NoSuchElementException:
                        try:
                            value_field = element.find_element(By.CSS_SELECTOR, 'textarea[name$=".dataValue"]')
                            value_field.clear()
                            value_field.send_keys(str(new_value))
                            print(f"✅ {col_name} 값 입력 완료: {new_value}")
                        except NoSuchElementException:
                            print(f"⚠️ {col_name} 값 필드 없음")
                    processed_columns.add(col_name)
                    print(f"✅ {col_name} 처리 완료 (처리된 컬럼 수: {len(processed_columns)}/{total_columns_to_process})")
                except StaleElementReferenceException:
                    print(f"⚠️ {col_name} StaleElement 발생, 재탐색 필요")
            else:
                print(f"⚠️ {col_name} 찾을 수 없음")

            # 모든 컬럼 처리 완료 시 종료
            if len(processed_columns) == total_columns_to_process:
                print("✅ 모든 엑셀 데이터 처리 완료, 루프 종료")
                break

    except Exception as e:
        print(f"⚠️ 데이터 붙여넣기 실패: {e}")
    finally:
        driver.switch_to.default_content()
    try:
        # 수정 버튼 클릭 및 화면 축소
        edit_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//button[contains(@class, "ui mini primary button") and text()="수정"]')))
        edit_button.click()
        print("✅ 수정 버튼 클릭 완료!")
        time.sleep(1)
        driver.execute_script("document.body.style.zoom='25%'")
        print("✅ 웹 화면 25% 축소 완료!")
        time.sleep(2)

        # iframe 또는 기본 컨텍스트 확인
        driver.switch_to.default_content()
        context_found = False
        try:
            WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CLASS_NAME, "_columnBorder-edit-modal_1rntr_47")))
            print("🔍 기본 콘텐츠에서 요소 발견!")
            context_found = True
        except TimeoutException:
            iframes = driver.find_elements(By.TAG_NAME, "iframe")
            for i, frame in enumerate(iframes):
                driver.switch_to.frame(frame)
                try:
                    WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CLASS_NAME, "_columnBorder-edit-modal_1rntr_47")))
                    print(f"🔍 iframe {i}에서 요소 발견!")
                    context_found = True
                    break
                except TimeoutException:
                    driver.switch_to.default_content()
            if not context_found:
                print("⚠️ 모든 컨텍스트에서 요소를 찾지 못함")
                return
        
        # 처리된 컬럼을 추적할 set (중복 방지)
        processed_columns = set()
        
        # 데이터 입력
        for col_name, _, new_value in data:
            if col_name in processed_columns:
                print(f"ℹ️ {col_name} 이미 처리됨, 건너뜀")
                continue

            print(f"🔍 처리 중: {col_name}")
            try:
                elements = WebDriverWait(driver, 15).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "_columnBorder-edit-modal_1rntr_47")))
                found = False
                for idx, element in enumerate(elements):
                    print(f"idx: {idx+1}, element: {element}")
                    try:
                        col_input = element.find_element(By.CSS_SELECTOR, 'input[type="text"][name$=".columnName"]')
                        current_col_name = col_input.get_attribute("value")
                        if current_col_name == col_name:
                            found = True
                            processed_columns.add(col_name)  # 중복 방지를 위해 set에 추가
                            driver.execute_script("arguments[0].scrollIntoView(true);", element)
                            try:
                                ace_editor = element.find_element(By.CSS_SELECTOR, "div.ace_editor")
                                driver.execute_script("arguments[0].click();", ace_editor)
                                
                                set_ace_editor_value(driver, ace_editor, new_value)
                            except NoSuchElementException:
                                try:
                                    value_field = element.find_element(By.CSS_SELECTOR, 'textarea[name$=".dataValue"]')
                                    value_field.clear()
                                    
                                    value_field.send_keys(str(new_value))
                                    print(f"✅ {col_name} 값 입력 완료: {new_value}")
                                except NoSuchElementException:
                                    print(f"⚠️ {col_name} 값 필드 없음")
                            break  # 컬럼 처리 완료 후 루프 종료
                    except StaleElementReferenceException:
                        print(f"⚠️ {col_name} StaleElement 발생 (인덱스 {idx}), 재탐색")
                        continue
                if not found:
                    print(f"⚠️ {col_name} 찾을 수 없음")
            except TimeoutException:
                print(f"⚠️ {col_name} 요소 탐색 시간 초과")

    except Exception as e:
        print(f"⚠️ 데이터 붙여넣기 실패: {e}")
        driver.switch_to.default_content()

# 복사할 대상 입력
name = ""
stage = ""
csv_path = "C:/엑셀/Ranking_1 (2).csv"  #랭킹 데이터 
b2_value = "j"+"1" #클론 아이디
#엑셀 행번호
ranking = [1] # 복사할 랭킹 순위
size =len(ranking)

type = ["TREASURE_DATA", "GROWTH3_DATA", "BASE_DATA"]
#f"C:\\엑셀\\복사\\{name}{stage}.xlsx"

# ✅ 메인 실행
if __name__ == "__main__":
    for a in range(size):
        get_csv_value(ranking[a]-1, 2)
        for i in range(3):  # 0부터 2까지 3번 반복
            excel_filename = rf"C:\엑셀\복사\{type[i]}\슈바니.xlsx"  # type[2]는 "C"
            print(f"🔍 현재 작업 디렉토리: {os.getcwd()}")
            if not os.path.exists(excel_filename):
                print(f"⚠️ 파일이 존재하지 않습니다: {excel_filename}")
                # exit()  # 주석 해제 시 프로그램 종료

            # 아래 함수들은 정의가 필요합니다
            data = load_excel_data(excel_filename)
            if not data:
                print("⚠️ 데이터 로드 실패로 인해 프로그램을 종료합니다.")
                # exit()

            driver1 = setup_browser(0)
            try:
                login_to_console(driver1)
                navigate_to_page(driver1)
                confirm_click(driver1)
                execute_tasks(driver1, ranking[0]-1, 2, type[i])
                paste_data(driver1, data)
            except KeyboardInterrupt:
                print("⚠️ 사용자가 프로그램을 중단했습니다.")
            finally:
                print("✅ 브라우저가 종료되었습니다.")
                # driver1.quit()  # 필요 시 주석 해제



🔹 0행 3열 값 (출력용): 슈바니
🔍 현재 작업 디렉토리: c:\JJG\Start\sjdj
✅ 엑셀 데이터 로드 완료! 경로: C:\엑셀\복사\TREASURE_DATA\슈바니.xlsx
✅ 로그인 완료!
✅ 페이지 로드 완료!
🔍 '확인' 버튼 탐색 중...
⚠️ 확인 버튼을 찾거나 클릭하지 못했습니다: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF71BA8CF45+75717]
	GetHandleVerifier [0x00007FF71BA8CFA0+75808]
	(No symbol) [0x00007FF71B858F9A]
	(No symbol) [0x00007FF71B8AF4C6]
	(No symbol) [0x00007FF71B8AF77C]
	(No symbol) [0x00007FF71B902577]
	(No symbol) [0x00007FF71B8D73BF]
	(No symbol) [0x00007FF71B8FF39C]
	(No symbol) [0x00007FF71B8D7153]
	(No symbol) [0x00007FF71B8A0421]
	(No symbol) [0x00007FF71B8A11B3]
	GetHandleVerifier [0x00007FF71BD8D71D+3223453]
	GetHandleVerifier [0x00007FF71BD87CC2+3200322]
	GetHandleVerifier [0x00007FF71BDA5AF3+3322739]
	GetHandleVerifier [0x00007FF71BAA6A1A+180890]
	GetHandleVerifier [0x00007FF71BAAE11F+211359]
	GetHandleVerifier [0x00007FF71BA95294+109332]
	GetHandleVerifier [0x00007FF71BA95442+109762]
	GetHandleVerifier [0x00007FF71BA7BA59+4825]
	BaseThreadInitThunk [0x00007FF